In [1]:
import sqlite3
import argparse
import os 
from collections import defaultdict
import numpy as np
from tqdm import tqdm
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import ast
from datetime import datetime
from flai.util.Time import Time

In [2]:
treedb = sqlite3.connect("../in/time_resolved_entropy/tree_H3N2_14_02.db", isolation_level = None)


In [3]:
df_tree = pd.read_sql_query("SELECT nid, pid, rcluster, clade, collection_date, region, multiplicity, name, epi, mutations, collection_coordinate, leaf_count, submission_coordinate, db_sequence FROM Tree",treedb)


In [4]:
df_sequence = pd.read_sql_query("SELECT hashed_sequence, gene_aa_sequence FROM Sequence",treedb)


In [5]:
isolates = df_tree[df_tree['epi'].str.contains("EPI_ISL")]


In [10]:
### sliding it by a month ###############

window_size = 365  # number of days in each window
start_date = Time.date_to_coordinate("1968-01-01")
end_date = Time.date_to_coordinate("2022-12-31")
step = 30  # number of days to slide the window

##calculating entropy

windows = []


for t in tqdm(range(start_date, end_date - window_size + 1, step)):
    window_start = t
    window_end = t + window_size
    window = isolates.loc[(isolates["collection_coordinate"] >= window_start) &
                          (isolates["collection_coordinate"] <= window_end)]

    pos2aa2count = defaultdict(lambda:defaultdict(lambda: 0.))
    for isolate in window.itertuples():
        # Get gene aa sequence for isolate and gene of interest
        gene_aa_seq = ast.literal_eval(str(df_sequence[df_sequence["hashed_sequence"]==isolate.db_sequence]["gene_aa_sequence"].values[0]))
        gene_aa = gene_aa_seq['HA']

        # Get parent aa sequence for isolate and gene of interest
        pid = isolate.pid
        parent_aa_seq = ast.literal_eval(str(df_sequence[df_sequence["hashed_sequence"]==df_tree.loc[df_tree["nid"]==pid]["db_sequence"].values[0]]["gene_aa_sequence"].values[0]))
        # Update count of amino acids at each position in the selected gene of interest
        for i, aa in enumerate(gene_aa):
            #print(i)
            if aa in ["?", "-"]:
                # Replace missing/ambiguous amino acid with corresponding amino acid in parent node
                parent_aa = parent_aa_seq['HA'][i]
                pos2aa2count[i+1][parent_aa] += 1.
            else:
                # Update count of amino acid at current position for current isolate
                pos2aa2count[i+1][aa] += 1.;


    # Calculate entropy
    pos2entropy = defaultdict(lambda: 0.0)
    for pos in pos2aa2count:
        P_aa = np.array([pos2aa2count[pos][aa] for aa in pos2aa2count[pos]]) / sum([pos2aa2count[pos][aa] for aa in pos2aa2count[pos]])
        num = sum([pos2aa2count[pos][aa] for aa in pos2aa2count[pos]])
        pos2entropy[pos] = np.sum([-p *np.log(p) for p in P_aa])
        # pos2_renyientropy[pos] = -np.log( np.dot(P_aa, np.dot(sim_matrix,P_aa) ) )
        
    # Convert the entropy dictionary to a DataFrame
    df_entropy = pd.DataFrame.from_dict(pos2entropy, orient='index', columns=['Entropy'])
    df_entropy.index.name = 'Position'

    # Extract the start and end dates of the current window
    window_start_date = Time.coordinate_to_date(window_start).strftime('%Y-%m-%d')
    window_end_date = Time.coordinate_to_date(window_end).strftime('%Y-%m-%d')

    # Generate the CSV file name
    file_name = f'HA_entropy_{window_start_date}_{window_end_date}.csv'

    # Save the DataFrame to a CSV file
    output_folder = "../out/csv_one_month_slide_H3N2/"
    df_entropy.to_csv(output_folder + file_name) 

    
    
    


100%|██████████| 658/658 [24:43<00:00,  2.25s/it]
